# Lucas Pierru (PIEL14069708) - Équipe 16 - Laboratoire 2 - GPA671

# Exercice 1

## Question 1

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


X, y = make_classification(n_samples=1000, n_features=2, n_informative=1, n_redundant=0, n_clusters_per_class=1, random_state=42, shift=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=42) # On sépare nos données de tel sorte
# à avoir un test set correspondant à 80% des données

In [ ]:
# On affiche nos données d'entrainement
for i in range (len(y_train)):
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, cmap=plt.cm.Paired)

## Question 2

In [ ]:
class LinearSVM(object):
    def __init__(self, lambd: float, k: int, T: int) -> None:
        """
        Classificateur SVM entrainé avec l'algorithme PEGASOS.
        
        Parameters
        ----------
        lambd : float
            Paramètre de compromis entre la maximisation de la marge et le respect des contraintes du SVM.
        k : int
            Nombre d'exemples aléatoirement choisis parmis les données d'entrainement dans chaque itération.
        T : int
            Nombre d'itérations d'entrainement.

        """
        self.lambd = lambd
        self.k = k
        self.T = T
        self.w = np.random.rand(1,2)
        
        self.w = normalize_w(self.w, self.lambd)
        self.b = 0

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Entrainement du SVM.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y_train : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        
        Ap = np.empty((0,3))
        grad_ft_w = np.zeros((1,2))
        grad_ft_b = 0
        
        for t in range (1,self.T+1):
            rands = np.random.choice(len(X_train), self.k, replace = False)

            At = np.array(X_train[rands])
            At = np.append(At, np.reshape(y_train[rands],(self.k,1)), axis = 1)
            score = self.decision_function(At[:,:2])
            
            Ap = At[:,2] * score < 1
            Ap = At*np.transpose(Ap)
            Ap = Ap[~np.all(Ap == 0, axis=1)]
            
            grad_ft_w = self.lambd * self.w -(1/self.k)*np.sum(Ap[:,:2]*np.reshape(Ap[:,2],(len(Ap),1)),axis = 0)
            grad_ft_b = -(1/self.k)*np.sum(np.reshape(Ap[:,2],(len(Ap),1)),axis = 0)
            
            lr = 1/(self.lambd*t)
            
            w_temp = self.w - lr*grad_ft_w
            self.b = self.b - lr*grad_ft_b
            self.w = min(1,1/(np.sqrt(self.lambd)*np.linalg.norm(w_temp)))*w_temp

    
    def decision_function(self, X: np.ndarray) -> np.ndarray:
        """
        Évaluation de la fonction de décision du SVM linéaire : w^T x + B.

        Parameters
        ----------
        X_train : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
            
        Returns
        -------
        scores : np.ndarray
            Score(s) pour chaque donnée d'entrée. Taille : (# exemples,).
        
        """
        scores = self.w@np.transpose(X) +self.b
        return scores
        
        pass

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du SVM.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée. Taille : (# exemples,).

        """
        scores = self.decision_function(X)
        
        y_pred = scores>0
        y_pred = 2*y_pred-1
        
        return y_pred

# Cette fonction vérifie que w respecte bien les conditions demandés dans l'énoncé 
def normalize_w(w, lam):

    if ((np.linalg.norm(w))>(1/(np.sqrt(lam)))):

            w =  w/ (np.sqrt(lam)*np.linalg.norm(w))
            
            return w

    else:
        return w

## Question 3

In [ ]:
lamb = [0.0001,0.001,0.01,0.1,1,10,100]
k = [1,2,5,10,20,50,100,200]
T = 1000
max = 0
Tef = np.zeros((len(lamb),len(k)))
# On itère parmis tout nos lambda et nos k pour trouver les meilleurs hype-paramètres
for i in range (len(lamb)):
    for j in range (len(k)):
        
        L = LinearSVM(lamb[i],k[j],T)
        L.fit(X_train, (y_train*2)-1) # On s'asssure également d'avoir des y entre -1 et 1
        Te = L.predict(X_val)*((y_val*2)-1)
        Te = (Te+1)/2 # On calcule notre Te
        
        Tef[i][j] = np.sum(Te)/len(Te[0])

        if Tef[i][j] > max:
            max = Tef[i][j]
            kmax = k[j]
            lambmax = lamb[i]
            
print("Le meilleur k est :",kmax," avec lambda :", lambmax," et Te :", max)

In [ ]:
fig = plt.figure(figsize=(12,8))
# On affiche notre graphique du taux d'exactitude en fonction de k et lambda
plt.imshow(Tef, interpolation='bilinear', extent = [np.min(k),np.max(k),np.max(lamb),np.min(lamb)], vmin= 0, vmax = 1, aspect='auto')
plt.title("Taux d'exactitude")
plt.colorbar()
plt.xlabel('k')
plt.ylabel('Lambda')
plt.yscale('log')
plt.show()

## Question 4

In [ ]:
# On créer un modèle avec le meilleur lambda et k pour voir la frontière de décision.
L = LinearSVM(lambmax,kmax,T)
L.fit(X_train, (y_train*2)-1)

Te = L.predict(X_val)*((y_val*2)-1)
Te = (Te+1)/2
        
Tef = np.sum(Te)/len(Te[0])

print("Taux d'exactitude : ",Tef)


decision_function = L.decision_function(X_val)

# On définit nos vecteurs de support comme les points étant sur la marge
support_vector_indices = np.where((np.abs(decision_function[0]) <= 1 + 1e-2)&(np.abs(decision_function[0]) >= 1 - 1e-2))[0]
support_vectors = X_val[support_vector_indices]

plt.scatter(X_val[:, 0], X_val[:, 1], c=y_val, s=30, cmap=plt.cm.Paired)
        
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(
    np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
)
Z = L.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(
    xx,
    yy,
    Z,
    colors="k",
    # On affiche les courbe pour la frontière de décision = -1, 0 et 1
    levels=[-1, 0, 1],
    alpha=0.5,
    linestyles=["--", "-", "--"],
)
# On affiche nos vecteurs de supports avec des cercles
plt.scatter(
        support_vectors[:, 0],
        support_vectors[:, 1],
        s=100,
        linewidth=0.5,
        facecolors="none",
        edgecolors="k",
    )

## Question 5

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42) # On va cette fois afficher des points qui forment des lunes
X, X_val, y, y_val = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
for i in range (len(y)):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

In [ ]:
# On répète la même opération
L2 = LinearSVM(lambmax,kmax,T)
L2.fit(X, (y*2)-1)

decision_function = L2.decision_function(X_val)

Te = L2.predict(X_val)*((y_val*2)-1)
Te = (Te+1)/2
        
Tef = np.sum(Te)/len(Te[0])

print("Taux d'exactitude : ",Tef)

support_vector_indices = np.where((np.abs(decision_function[0]) <= 1 + 1e-2) & (np.abs(decision_function[0]) >= 1 - 1e-2))[0]
support_vectors = X_val[support_vector_indices]

plt.scatter(X_val[:, 0], X_val[:, 1], c=y_val, s=30, cmap=plt.cm.Paired)
        
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(
    np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
)

Z = L2.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(
    xx,
    yy,
    Z,
    colors="k",
    levels=[-1, 0, 1],
    alpha=1,
    linestyles=["--", "-", "--"],
)

plt.scatter(
        support_vectors[:, 0],
        support_vectors[:, 1],
        s=100,
        linewidth=0.5,
        facecolors="none",
        edgecolors="k",
    )


# Exercice 2

## Question 1

In [ ]:
class KernelSVM(object):
    def __init__(self, lambd: float, k: int, T: int) -> None:
        """
        Classificateur SVM avec noyau entrainé avec l'algorithme PEGASOS.
        
        Parameters
        ----------
        lambd : float
            Paramètre de compromis entre la maximisation de la marge et le respect des contraintes du SVM.
        k : int
            Nombre d'exemples aléatoirement choisis parmis les données d'entrainement dans chaque itération.
        T : int
            Nombre d'itérations d'entrainement.

        """
        self.lambd = lambd
        self.k = k
        self.T = T
        
    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Entrainement du SVM.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y_train : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        self.y_train = y_train
        self.X_train = X_train
        self.beta = np.zeros((len(X_train),self.T))
        self.gamma = 1/(2*np.var(X_train))
        
        for t in range (1,self.T):
            rands = np.random.choice(len(X_train), self.k, replace = False)

            At = np.array(rands)
    
            for n in range (len(X_train)):
                if (n in At) :
                    if (((y_train[n]/(self.lambd*t))*np.sum(self.beta[:,t-1]*y_train*self.K(X_train, X_train[n])))<1):
                        self.beta[n][t] = self.beta[n][t-1] + 1
                    else:
                        self.beta[n][t] = self.beta[n][t-1]  
                else:
                    self.beta[n][t] = self.beta[n][t-1]
                    
        self.alpha = (1/(self.lambd*self.T))*self.beta[:,self.T-1]
        self.S = np.where(self.alpha != 0)[0]
        self.support_vector = X_train[self.S]
        biais = np.zeros(len(self.S))
        
        for n in range (len(self.S)):

            biais[n] = y_train[self.S[n]]-np.sum(self.alpha*y_train*self.K(X_train[self.S[n]],X_train))

        self.b = (1/len(self.S))*np.sum(biais)   
    
    def decision_function(self, X: np.ndarray) -> np.ndarray:
        """
        Évaluation de la fonction de décision du SVM avec noyau.

        Parameters
        ----------
        X_train : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
            
        Returns
        -------
        scores : np.ndarray
            Score(s) pour chaque donnée d'entrée. Taille : (# exemples,).
        
        """
        len_X = len(X)
        scores = np.zeros(len_X)
        for i in range(len_X):
            scores[i] = np.sum(self.alpha[self.S]*self.y_train[self.S]*self.K(X[i], self.X_train[self.S]))+self.b

        return scores

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du SVM.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée. Taille : (# exemples,).

        """
        scores = self.decision_function(X)
        y_pred = (scores>0)*1
        return y_pred
    # Fonction qui calcul notre K
    def K(self, Xn, Xm):
        k = np.exp(-self.gamma*(self.norm(Xn,Xm))**2)
        return k
    # Fonction qui calcul notre norme
    def norm(self, Xn, Xm):
        return np.sqrt(np.sum((Xn-Xm)**2,axis=1))



Bloc de test avec des valeurs arbitraire de k et lambda

In [ ]:
kn = KernelSVM(0.01, 20, 10)
y_signed = 2*y -1
kn.fit(X, y_signed)
y_pred = kn.predict(X_val)

te = np.sum((y_val == y_pred)*1)/len(y_val)

print(te)

## Question 2

In [ ]:
lamb = [0.0001,0.001,0.01,0.1,1,10,100]
k = [1,2,5,10,20,50,100,200]
T = 1000
max = 0
kmax = 0
lambmax = 0

Tef = np.zeros((len(lamb),len(k)))
# On procède au mêmes étapes qu'à l'exercice 1
for i in range (len(lamb)):
    for j in range (len(k)):
        kn = KernelSVM(lamb[i],k[j],T)
        kn.fit(X, y_signed)
        y_pred = kn.predict(X_val)
        te = np.sum((y_val == y_pred)*1)/len(y_val)
        
        Tef[i][j] = te
        
        if te > max:
            max = te
            kmax = k[j]
            lambmax = lamb[i]

        print("Taux d'exactitude : ",Tef[i][j],"Avec k :",k[j],"et lambda :",lamb[i])
print("Le meilleur k est :",kmax,"avec lambda :", lambmax,"et Te :", max)

In [ ]:
# On affiche également notre taux d'exactitude en fonction de k et lambda
fig = plt.figure(figsize=(12,8))
plt.imshow(Tef, interpolation='bilinear', extent = [np.min(k),np.max(k),np.max(lamb),np.min(lamb)], vmin= 0, vmax = 1, aspect='auto')
plt.title("Taux d'exactitude")
plt.colorbar()
plt.xlabel('k')
plt.ylabel('Lambda')
plt.yscale('log')
plt.show()

## Question 3

In [ ]:
# Comme à l'exercice précédent on va afficer le graphique du meilleur k et lamda
KF = KernelSVM(lambmax,kmax,T)
KF.fit(X, y_signed)
decision_function = KF.decision_function(X_val)
y_pred = KF.predict(X_val)

te1 = np.sum((y_val == y_pred)*1)/len(y_val)

print("Taux d'exactitude : ",te1,"Avec k :",kmax,"et lambda :",lambmax)

support_vectors = KF.support_vector # Cette fois-ci on utilisera les valeurs de S comme indices des vecteurs supports

plt.scatter(X_val[:, 0], X_val[:, 1], c=y_val, s=30, cmap=plt.cm.Paired)

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(
    np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
)

Z = KF.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(
    xx,
    yy,
    Z,
    colors="k",
    levels=[-1, 0, 1],
    alpha=0.5,
    linestyles=["--", "-", "--"],
)

plt.scatter(
        support_vectors[:, 0],
        support_vectors[:, 1],
        s=50,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )

## Question 4 (Vérification avec sklearn)

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

clf = SVC(C = 1/(4*lambmax))
clf.fit(X,y)
y_pred = clf.predict(X_val)

print("Accuracy:",metrics.accuracy_score(y_val, y_pred))

decision_function = clf.decision_function(X_val)
support_vectors = clf.support_vectors_

plt.scatter(X_val[:, 0], X_val[:, 1], c=y_val, s=30, cmap=plt.cm.Paired)
        
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(
    np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
)
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(
    xx,
    yy,
    Z,
    colors="k",
    levels=[-1, 0, 1],
    alpha=0.5,
    linestyles=["--", "-", "--"],
)
plt.scatter(
        support_vectors[:, 0],
        support_vectors[:, 1],
        s=50,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )

On peut voir, tout d'abord, que les résultats obtenus par notre classe et l'algorithme PEGASOS sont très similaire à ceux obtenus par la librairie sklearn. Notre frontière de décision et nos marges sont très ressemblante avec la vérification dans sklearn. On peut voir par contre, dans notre graphique que les vecteurs de supports se situe un peu partout dans le celui-ci y compris en dehors des marges. Dans le graphique réalisé à partir de sklearn, on peut voir que les vecteurs de supports sont restraint à l'intérieur des marges. 

$C$ et $\lambda$ sont tous les deux des paramètre de régularisation qui s'occupent de trouver un bon compromis entre la maximisation des marges et la minimisation des erreur de classification. On choisit $C=\frac{1}{4\lambda}$ car nos graphiques se ressemble beaucoup avec cette condition.

# Exercice 3

## Question 1

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris(return_X_y=True)

## Question 2

In [ ]:
# On sépare de manière balancée nos données
X_train, X_test, y_train, y_test = train_test_split(data[0], data[1],test_size=0.5, random_state=0)

## Question 3

In [ ]:
class kNN(object):
    def __init__(self, k: int) -> None:
        """
        Classificateur kNN.
        
        Parameters
        ----------
        k : int
            Nombre de plus proches voisins utilisés pour classifier un nouveau point.

        """
        self.k = k

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Chargement des données.
        
        Parameters
        ----------
        X : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        self.X_train = X
        self.y_train = y

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        probabilities : np.ndarray
            Vecteur(s) de probabilités prédits pour chaque données d'entrée. Taille : (# exemples, # classes).

        """
        self.d = np.zeros((len(X), len(self.X_train)))
        k_nn_values = np.zeros((len(X),3))
        
        for i in range(len(X)):
            for j in range(len(self.X_train)):
                self.d[i][j] = np.sqrt(np.sum((X[i]-self.X_train[j])**2))
            for k in range(self.k):
                
                k_nn_values[i][self.y_train[np.argpartition(self.d[i], self.k)[:self.k]][k]] += 1
            
        probabilities = k_nn_values/self.k
        return probabilities

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée.. Taille : (# exemples,).

        """
        prob = self.predict_proba(X)
        y_pred = np.argmax(prob, axis=1)
        
        return y_pred

In [ ]:
knn = kNN(3)

knn.fit(X_train,y_train)

prob = knn.predict_proba(X_test)

In [ ]:
y_pred = knn.predict(X_test)

## Question 4

In [ ]:
te = np.zeros(10)
k_range = range(1,11)
# On calcul nos taux d'exactitude pour k allant de 1 à 10
for k in k_range:
    knn = kNN(k)
    knn.fit(X_train,y_train)
    y_pred = knn.predict(X_test)
    
    y_diff = y_pred - y_test
    
    y_diff = (y_diff == 0)*1
    
    te[k-1] = np.sum(y_diff)/len(y_diff)
    
    print("Pour k = ",k,", la précision est de :",round(te[k-1]*100,2),"%")

plt.plot(k_range, te , color = 'red') 

On peut voir avec que plus on augmente le nombre de voisins plus le taux d'exactitude a tendance à augmenter avec le meilleur taux pour k = 9.
Notre dataset est assez petit, donc cela fonctionne assez bien mais on sait qu'avec un plus grand nombre de données, l'apprentissage peut devenir lent. 

### On verifie avec le modèle dans sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k_range = range(1,11)

scores = {}
scores_list = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores[k] = metrics.accuracy_score(y_test, y_pred)
    scores_list.append(metrics.accuracy_score(y_test, y_pred))
    print("Pour k = ",k,", la précision est de :",round(scores[k]*100,2),"%")

# Exercice 4 (Bonus)

## Question 1

In [ ]:
class NaivesBayes(object):
    def __init__(self) -> None:
        """
        Classificateur Bayesien.

        """
        pass

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Chargement des données.
        
        Parameters
        ----------
        X : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        self.X_train = X
        self.y_train = y
        
        self.values = np.unique(self.y_train)
        for val in self.values:
            exec(f'idx_{val} = np.where(self.y_train == val)')
            exec(f'self.X_train_{val} = self.X_train[idx_{val}]')
            exec(f'self.X_train_{val}_mean = np.mean(self.X_train_{val}, axis=0)')
            exec(f'self.X_train_{val}_std = np.std(self.X_train_{val}, axis=0)')
            exec(f'self.prob_{val} = len(idx_{val}[0])/len(self.y_train)')
            

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        probabilities : np.ndarray
            Vecteur(s) de probabilités prédits pour chaque données d'entrée. Taille : (# exemples, # classes).

        """

        probabilities = np.zeros((len(X),len(self.values)))
        
        for val in self.values:
            exec(f'probabilities_{val} = proba_condition(X, self.X_train_{val}_mean, self.X_train_{val}_std)')
            exec(f'probabilities[:,val] = self.prob_{val}*np.prod(probabilities_{val}, axis=1)')
        
        return probabilities

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée.. Taille : (# exemples,).

        """
        prob = self.predict_proba(X)
        y_pred = np.argmax(prob, axis=1)
        
        return y_pred
    
def proba_condition(x,mean,std):
    return (1/(std*np.sqrt(2*np.pi)))*np.exp(-(x-mean)**2/(2*std**2))

## Question 2

In [ ]:
nb = NaivesBayes()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
y_diff = y_pred - y_test
    
y_diff = (y_diff == 0)*1

te = np.sum(y_diff)/len(y_diff)

print("La précision est de :",round(te*100,2),"%")

### On vérifie avec sklearn

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_sk = GaussianNB(priors = None)
model_sk.fit(X_train,y_train)

y_pred = model_sk.predict(X_test)

print("La précision est de :",round(metrics.accuracy_score(y_test, y_pred)*100,2),"%")

On obtient un taux qui est supérieur à la plupart des valeurs obtenues avec le KNN mais quand même inférieur au meilleur. On reste tout de fois dans les mêmes ordres de grandeurs. Le classificateur Bayesien naïf est meilleurs si nos données sont en plusieurs dimensions et si elle sont en grandes quantités. Il fera les calculs plus rapidement et donnera une meilleure précision au final. Il est aussi bien plus simple à implémenter que le K-NN.